In [1]:
import rpy2.ipython
rpy2.ipython.load_ipython_extension(get_ipython())

In [2]:
%%R
library(recount3)
library(tidyverse)

R[write to console]: Loading required package: SummarizedExperiment

R[write to console]: Loading required package: MatrixGenerics

R[write to console]: Loading required package: matrixStats

R[write to console]: 
Attaching package: ‘MatrixGenerics’


R[write to console]: The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiff

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ lubridate::%within%() masks IRanges::%within%()
✖ dplyr::collapse()     masks IRanges::collapse()
✖ dplyr::combine()      masks Biobase::combine(), BiocGenerics::combine()
✖ dplyr::count()        masks matrixStats::count()
✖ dplyr::desc()         masks IRanges::desc()
✖ tidyr::expand()       masks S4Vectors::expand()
✖ dplyr::filter()       masks stats::filter()
✖ dplyr::first()        masks S4Vectors::first()
✖ dplyr::lag()          masks stats::lag()
✖ ggplot2::Position()   masks BiocGenerics::Position(), base::Position()
✖ purrr::reduce()       masks GenomicRanges::reduce(), IRanges::reduce()
✖ dplyr::rename()       masks S4Vectors::rename()


In [3]:
%%R
# 1. Cargar metadatos de GTEx (project="gtex")
url <- "https://storage.googleapis.com/adult-gtex/annotations/v8/metadata-files/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt"
download.file(url, "GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt")
metadata <- read.delim("GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt")
head(metadata)


R[write to console]: probando la URL 'https://storage.googleapis.com/adult-gtex/annotations/v8/metadata-files/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt'

R[write to console]: Content type 'text/plain'
R[write to console]:  length 11512258 bytes (11.0 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: 

                         SAMPID SMATSSCR SMCENTER
1      GTEX-1117F-0003-SM-58Q7G       NA       B1
2      GTEX-1117F-0003-SM-5DWSB       NA       B1
3      GTEX-1117F-0003-SM-6WBT7       NA       B1
4 GTEX-1117F-0011-R10a-SM-AHZ7F       NA   B1, A1
5 GTEX-1117F-0011-R10b-SM-CYKQ8       NA   B1, A1
6      GTEX-1117F-0226-SM-5GZZ7        0       B1
                                      SMPTHNTS SMRIN           SMTS
1                                                 NA          Blood
2                                                 NA          Blood
3                                                 NA          Blood
4                                                 NA          Brain
5                                                7.2          Brain
6 2 pieces, ~15% vessel stroma, rep delineated   6.8 Adipose Tissue
                         SMTSD SMUBRID SMTSISCH SMTSPAX SMNABTCH
1                  Whole Blood 0013756     1188      NA BP-38516
2                  Whole Blood 0013756     1

In [4]:
%%R
# 2. Filtrar solo las columnas deseadas
metadata_GTEX <- metadata %>% mutate(DONOR_ID = sub("^(([^-]+-[^-]+)).*", "\\1", SAMPID)) %>% 
    filter(str_starts(DONOR_ID, "GTEX-"))
head(metadata_GTEX)


                         SAMPID SMATSSCR SMCENTER
1      GTEX-1117F-0003-SM-58Q7G       NA       B1
2      GTEX-1117F-0003-SM-5DWSB       NA       B1
3      GTEX-1117F-0003-SM-6WBT7       NA       B1
4 GTEX-1117F-0011-R10a-SM-AHZ7F       NA   B1, A1
5 GTEX-1117F-0011-R10b-SM-CYKQ8       NA   B1, A1
6      GTEX-1117F-0226-SM-5GZZ7        0       B1
                                      SMPTHNTS SMRIN           SMTS
1                                                 NA          Blood
2                                                 NA          Blood
3                                                 NA          Blood
4                                                 NA          Brain
5                                                7.2          Brain
6 2 pieces, ~15% vessel stroma, rep delineated   6.8 Adipose Tissue
                         SMTSD SMUBRID SMTSISCH SMTSPAX SMNABTCH
1                  Whole Blood 0013756     1188      NA BP-38516
2                  Whole Blood 0013756     1

In [8]:
%%R
nrow(metadata)


[1] 22951


In [7]:
%%R
nrow(metadata_GTEX )

[1] 22734


In [9]:
%%R
metadata_gtex_subset <- metadata_GTEX  %>%
  select(SAMPID,SMTS,SMTSD,SMAFRZE,DONOR_ID) 

#head(metadata_gtex_subset)
write.csv(metadata_gtex_subset , file = "metadata_gtex.csv", row.names = FALSE)

In [10]:
%%R
nrow(metadata_gtex_subset )

[1] 22734


In [11]:
%%R
metadata_sub <- metadata_gtex_subset %>%
        filter(str_starts(SMAFRZE, "RNASEQ"))

nrow(metadata_sub)
head(metadata_sub)
write.csv(metadata_sub , file = "metadata.csv", row.names = FALSE)

In [12]:
%%R
nrow(metadata_sub)

[1] 17382


In [ ]:
%%R
# Para contar cuántas muestras hay por DONOR_ID y SMTS
df_summary <- metadata_sub %>%
  group_by(DONOR_ID, SMTS) %>%
  summarise(n = n()) %>%
  ungroup()

# Pivotar para tener los SMTS como columnas y valores el conteo
metadata_SMTS  <- df_summary %>%
  pivot_wider(names_from = SMTS, values_from = n, values_fill = 0)

metadata_SMTS 
write.csv(metadata_SMTS , file = "metadata_SMTS .csv", row.names = FALSE)

In [ ]:
%%R
# Para contar cuántas muestras hay por DONOR_ID y SMTS
df_summary <- metadata_sub %>%
  group_by(DONOR_ID, SMTSD) %>%
  summarise(n = n()) %>%
  ungroup()

# Pivotar para tener los SMTS como columnas y valores el conteo
metadata_SMTSD <- df_summary %>%
  pivot_wider(names_from = SMTSD, values_from = n, values_fill = 0)

metadata_SMTSD
write.csv(metadata_SMTSD , file = "metadata_SMTSD.csv", row.names = FALSE)

In [ ]:
%%R
# 5. Ver resultado
head(metadata_subset)

In [13]:
%%R
tab <- table(metadata_sub$DONOR_ID, metadata_sub$SMTS) # o df$SMTSD o una combinación

In [14]:
%%R
# Para combinar SMTS y SMTSD:
tab2 <- table(metadata_sub$DONOR_ID, paste(metadata_sub $SMTS, metadata_sub $SMTSD, sep = " - "))
head(tab2)
write.csv(tab2, file = "metadata_agrupada.csv", row.names = T)